In [1]:
import os
import time
import pickle
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import log_loss, roc_auc_score

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss, MSELoss
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

import sys
sys.path.insert(0, '../data/')
import janestreet

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

CACHE_PATH = './v08_pytorch_benchmark/'
NFOLDS = 5

In [3]:
feat_cols = [f'feature_{i}' for i in range(130)]
target_cols = ['action', 'action_1', 'action_2', 'action_3', 'action_4']
all_feat_cols = feat_cols.copy()
all_feat_cols.extend(['cross_41_42_43', 'cross_1_2'])
f_mean = np.load(f'{CACHE_PATH}/f_mean_online.npy')

# Prediction

In [4]:
class MLPModel(nn.Module):
    
    # training parameters
    epochs = 200
    label_smoothing = 1e-2
    learning_rate = 1e-3
    
    # model parameters
    hidden_units = [160, 160, 160]
    dropout_rates = [0.2, 0.2, 0.2, 0.2]
    num_columns = len(all_feat_cols)
    num_labels = len(target_cols)
    units = [num_columns] + hidden_units + [num_labels]
    
    def __init__(self):
        super(MLPModel, self).__init__()
        self.batch_norm = nn.ModuleList()
        self.dropout = nn.ModuleList()
        self.dense = nn.ModuleList()
        
        for i in range(len(self.units) - 1):
            self.batch_norm.append(nn.BatchNorm1d(self.units[i]))
            self.dropout.append(nn.Dropout(self.dropout_rates[i]))
            self.dense.append(nn.Linear(self.units[i], self.units[i + 1]))
        
        self.activation = nn.SiLU()
        
    def forward(self, x):
        for i in range(len(self.units) - 1):
            x = self.batch_norm[i](x)
            if i != 0:
                x = self.activation(x)
            x = self.dropout[i](x)
            x = self.dense[i](x)
        # no sigmoid
            
        return x

In [5]:
model_list = []
for _fold in range(NFOLDS):
    model = MLPModel()
    model_weights = f"{CACHE_PATH}/online_model{_fold}.pth"
    model.load_state_dict(torch.load(model_weights))
    model.eval()
    model_list.append(model)

In [6]:
env = janestreet.make_env()
env_iter = env.iter_test()

device = torch.device("cpu")

for (test_df, pred_df) in tqdm(env_iter):
    if test_df['weight'].item() > 0:
        x_tt = test_df.loc[:, feat_cols].values
        if np.isnan(x_tt.sum()):
            x_tt = np.nan_to_num(x_tt) + np.isnan(x_tt) * f_mean

        cross_41_42_43 = x_tt[:, 41] + x_tt[:, 42] + x_tt[:, 43]
        cross_1_2 = x_tt[:, 1] / (x_tt[:, 2] + 1e-5)
        feature_inp = np.concatenate((
            x_tt,
            np.array(cross_41_42_43).reshape(x_tt.shape[0], 1),
            np.array(cross_1_2).reshape(x_tt.shape[0], 1),
        ), axis=1)

        # torch_pred
        torch_pred = np.zeros((1, len(target_cols)))
        for model in model_list:
            torch_pred += model(torch.tensor(feature_inp, dtype=torch.float).to(device)).sigmoid().detach().cpu().numpy() / NFOLDS
        torch_pred = np.median(torch_pred)

        # tf_pred
        #tf_pred = np.median(np.mean([model(x_tt, training = False).numpy() for model in tf_models],axis=0))

        # avg
        #pred = torch_pred * 0.5 + tf_pred * 0.5
        pred = torch_pred

        pred_df.action = np.where(pred >= 0.5, 1, 0).astype(int)
    else:
        pred_df.action = 0
    env.predict(pred_df)

15219it [01:58, 128.15it/s]
